# DM-35166 Clean up annotation tags in SQuaSH
See also [annotation guidelines](https://confluence.lsstcorp.org/display/DM/SQuaSH+Annotations).

## Connect to the SQuaSH instance

In [ ]:
import asyncio
from aioinflux import InfluxDBClient

In [ ]:
INFLUXDB_URL = "influxdb-demo.lsst.codes"

See SQuaSH InfluxDB user credentials in SQuaRE 1Password

In [ ]:
import getpass
username = "squash"
password = getpass.getpass(prompt='Password for user `{}`: '.format(username))

## Query `chronograf-sandbox`

`chronograf-sandbox` is a copy of the `chronograf` database which contains the annotations data. The following query was used to create `chronograf-sandbox`:
```
> SELECT * INTO "chronograf-sandbox".autogen.:MEASUREMENT FROM "chronograf".autogen./.*/ GROUP BY *
```

In [ ]:
client = InfluxDBClient(host=INFLUXDB_URL, port=443, ssl=True, db="chronograf-sandbox", username=username, password=password, output="dataframe")

## Retrieve annotations, skip deleted ones

In [ ]:
df = await client.query(f"SELECT * FROM annotations WHERE deleted!=True")
df

## Clean up tag keys

### 1. Remove `brightSnrMin`, `Feature` and `nMinPhotRepeat` tag keys

In [ ]:
df=df.drop(columns=["brightSnrMin", "Feature", "nMinPhotRepeat"])
df.head()

### 2. Merge  `:Dataset:`, `ci_dataset`, `Dataset` into `Dataset`

In [ ]:
df["Dataset"] = df[[":Dataset:", "Dataset", "ci_dataset"]].apply(lambda x: x[0] or x[1] or x[2], axis=1)
df = df.drop(columns=[":Dataset:", "ci_dataset"])
df.head()

### 3. Merge `Pipeline`, `Pipeliine`, `pipeline`, `Pipelines` into `Pipeline`

In [ ]:
df["Pipeline"] = df[["Pipeline", "Pipeliine", "pipeline", "Pipelines"]].apply(lambda x: x[0] or x[1] or x[2] or x[3], axis=1)
df = df.drop(columns=["pipeline", "Pipeliine", "Pipelines"])
df.head()

### 4. Replace `faro=Pipeline` with `Pipeline=DRP`

In [ ]:
df.loc[df["faro"] == "Pipeline", "Pipeline"] = "DRP"
df = df.drop(columns=["faro"])
df.head()

## Clean up Dataset tag values 

### 5. Replace `CI-HiTS2015` and `HiTS2015` with `ap_verify_ci_hits2015`

In [ ]:
df.loc[df["Dataset"] == "CI-HiTS2015", "Dataset"] = "ap_verify_ci_hits2015"
df.loc[df["Dataset"] == "HiTS2015", "Dataset"] = "ap_verify_ci_hits2015"

# Clean up `Pipeline` tag values 

### 6. Replace `ap_pipe` with `AP`

In [ ]:
df.loc[df["Pipeline"] == "ap_pipe", "Pipeline"] = "AP"

### 7. Replace `hsc` with `Pipeline=DRP` and `Dataset=hsc`

In [ ]:
df.loc[df["Pipeline"] == "hsc", "Dataset"]="hsc"
df.loc[df["Pipeline"] == "hsc", "Pipeline"]="DRP"

### 8. Replace `faro, pipe_analysis` with `DRP`

In [ ]:
df.loc[df["Pipeline"] == "faro", "Pipeline"] = "DRP"
df.loc[df["Pipeline"] == "pipe_analysis", "Pipeline"] = "DRP"
df.head()

# Write annotations df to a new measurement in `chronograf-sandbox`

In [ ]:
# await client.write(df, measurement="annotations_clean", tag_columns=["Dataset", "Gen", "Instrument", "Pipeline", "id"])